# 8-9. 프로젝트 - Movielens 영화 추천 실습

---

## 0. 데이터 다운로드

In [1]:
# wget으로 데이터 다운로드
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2021-02-15 18:24:28--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M   198KB/s    in 35s     

2021-02-15 18:25:06 (163 KB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [2]:
# 다운받은 데이터를 작업디렉토리로 옮김
!mv ml-1m.zip ~/aiffel/recommendata_iu/data

# 작업디렉토리로 이동
!cd ~/aiffel/recommendata_iu/data

#압축 해제
!unzip ml-1m.zip

unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.


---

## 1. 데이터 준비와 전처리

In [3]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
ratings['count']
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movie_to_title = {k:v for k,v in zip(movies['movie_id'], movies['title'])}
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


---

## 2. 데이터 분석

In [8]:
# 검색을 쉽게하기 위해 영화 제목 문자열을 소문자로 변경
movies.head()
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


### ratings에 있는 유니크한 영화 개수

In [9]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [10]:
ratings['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [11]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
popular = movie_count.sort_values(ascending=False).head(30)
for i,c in zip(popular.index, popular):
    print(i, movie_to_title[i], c)

2858 American Beauty (1999) 3211
260 Star Wars: Episode IV - A New Hope (1977) 2910
1196 Star Wars: Episode V - The Empire Strikes Back (1980) 2885
1210 Star Wars: Episode VI - Return of the Jedi (1983) 2716
2028 Saving Private Ryan (1998) 2561
589 Terminator 2: Judgment Day (1991) 2509
593 Silence of the Lambs, The (1991) 2498
1198 Raiders of the Lost Ark (1981) 2473
1270 Back to the Future (1985) 2460
2571 Matrix, The (1999) 2434
480 Jurassic Park (1993) 2413
2762 Sixth Sense, The (1999) 2385
608 Fargo (1996) 2371
110 Braveheart (1995) 2314
1580 Men in Black (1997) 2297
527 Schindler's List (1993) 2257
1197 Princess Bride, The (1987) 2252
2396 Shakespeare in Love (1998) 2213
1617 L.A. Confidential (1997) 2210
318 Shawshank Redemption, The (1994) 2194
858 Godfather, The (1972) 2167
1265 Groundhog Day (1993) 2121
1097 E.T. the Extra-Terrestrial (1982) 2102
2997 Being John Malkovich (1999) 2066
2716 Ghostbusters (1984) 2051
296 Pulp Fiction (1994) 2030
356 Forrest Gump (1994) 2022
1240 

---

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [12]:
# 내가 좋아하시는 영화 5가지 추가(단, 이름은 꼭 데이터셋에 있는 것과 동일)
# Mask, Bicentennial Man, Patch Adams, Sawshank Redemption, Toy Story
my_favorite = [367, 3156, 2431, 318, 1]

# 'ryu'라는 user_id가 위 영화 별점을 5점씩 주었다고 가정
my_movielist = pd.DataFrame({'user_id': ['ryu']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['ryu']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10) 

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,ryu,367,5,NaN
1,ryu,3156,5,NaN
2,ryu,2431,5,NaN
3,ryu,318,5,NaN
4,ryu,1,5,NaN


In [13]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드(idx는 index의 약자)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['ryu']) 
print(movie_to_idx[367])

6039
458


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 생성
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,458,5,NaN
1,6039,897,5,NaN
2,6039,640,5,NaN
3,6039,157,5,NaN


---

## 4. CSR matrix를 직접 만들어 봅시다.

In [16]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

---

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

---

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
ryu, mask = user_to_idx['ryu'], movie_to_idx[367]
ryu_vector, mask_vector = als_model.user_factors[ryu], als_model.item_factors[mask]

print('슝=3')

슝=3


In [22]:
ryu_vector

array([-0.03250168, -0.33239338,  0.02734895,  0.5673748 ,  0.09915368,
        0.07313435, -0.44740173, -0.4184996 ,  0.41634357, -0.2986773 ,
        0.71028453, -0.8038868 , -0.10125992,  1.0061115 ,  0.1427423 ,
       -0.58152586,  0.8018944 ,  0.20970438, -0.30614984, -1.1540978 ,
       -0.1551555 , -0.5141593 , -0.6650801 ,  0.21219364, -0.5840908 ,
        0.11783487,  0.13690001, -0.8074244 ,  0.3745887 ,  0.00151416,
       -0.12426305,  0.4286694 , -0.33827177, -0.69006026,  0.17819351,
       -0.00434472, -0.19202563,  0.34331474,  0.04626601,  0.35347822,
        0.00213561, -0.06484187,  0.11712459,  0.1660869 , -0.20767196,
        0.3744184 , -0.49034697, -1.0507869 ,  0.9171917 ,  0.45001763,
       -0.38839564,  0.5529213 ,  0.7153131 ,  0.26832056,  0.07955246,
        0.7428877 ,  0.02190119,  0.06719157,  0.5519001 ,  0.07449573,
        0.07222997, -0.44733295, -0.2273484 ,  0.581011  , -0.42483258,
        0.03169282, -0.12765335, -0.5152176 ,  0.19356458,  0.56

In [23]:
mask_vector

array([-0.00776891,  0.0071392 , -0.02927606,  0.01287201,  0.00541914,
        0.00526082, -0.00844156, -0.0043164 ,  0.03768081, -0.00314535,
        0.01331532, -0.02173158,  0.01210265,  0.05259334, -0.01246516,
       -0.00582804,  0.03608559,  0.01199155, -0.00994703,  0.01101512,
       -0.01335099, -0.02342043, -0.0205692 ,  0.03340985, -0.02006583,
        0.00450475,  0.03007813, -0.01133481,  0.03066962,  0.00562671,
       -0.00085337,  0.01512153,  0.01171361,  0.00664077,  0.01281094,
        0.00195786,  0.00219103,  0.00125375, -0.00619223,  0.01739558,
        0.01738763, -0.00831392,  0.01898469,  0.01399371, -0.00512908,
        0.01184272,  0.01250628,  0.00056305,  0.04787404,  0.04238459,
       -0.0280182 , -0.00587696,  0.0259796 , -0.00863094,  0.01042614,
        0.02957048,  0.02615052, -0.00402154, -0.00325803,  0.01392642,
       -0.00055996,  0.01701979,  0.0011035 ,  0.00058445, -0.01757204,
       -0.00882494, -0.0045285 , -0.01638867, -0.00280655, -0.01

In [24]:
# 내가 선호하는 5가지 영화 중 하나
# ryu과 mask를 내적하는 코드
np.dot(ryu_vector, mask_vector)

0.40706974

In [25]:
# 내가 선호하지 않았던 영화 중 하나
# ryu과 screamers를 내적하는 코드
screamers = movie_to_idx[76]
screamers_vector = als_model.item_factors[screamers]
np.dot(ryu_vector, screamers_vector)

0.026015326

---

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.8266804),
 (4, 0.614899),
 (322, 0.57426465),
 (33, 0.56623703),
 (110, 0.53076977),
 (330, 0.4560852),
 (20, 0.45240897),
 (10, 0.43398336),
 (255, 0.42288846),
 (126, 0.39166278),
 (32, 0.37269238),
 (34, 0.3553485),
 (160, 0.34717628),
 (16, 0.34438887)]

In [28]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[[movie_to_title[idx_to_movie[i[0]]], i[0]] for i in similar_movie]

[['Toy Story (1995)', 40],
 ['Toy Story 2 (1999)', 50],
 ["Bug's Life, A (1998)", 4],
 ['Babe (1995)', 322],
 ['Aladdin (1992)', 33],
 ['Groundhog Day (1993)', 110],
 ['Lion King, The (1994)', 330],
 ['Pleasantville (1998)', 20],
 ['Beauty and the Beast (1991)', 10],
 ["There's Something About Mary (1998)", 255],
 ['Shakespeare in Love (1998)', 126],
 ['Hercules (1997)', 32],
 ['Mulan (1998)', 34],
 ['Forrest Gump (1994)', 160],
 ['Tarzan (1999)', 16]]

In [29]:
# 반복 확인을 위한 함수 생성
def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    similar_movie = [movie_to_title[i] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [30]:
# 내가 좋아하는 다른 영화 "쇼생크 탈출"로 확인
get_similar_movie(318)

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 "Schindler's List (1993)",
 'Good Will Hunting (1997)',
 'Pulp Fiction (1994)',
 'Fargo (1996)',
 'GoodFellas (1990)',
 'Dead Man Walking (1995)',
 'Saving Private Ryan (1998)',
 'Usual Suspects, The (1995)']

In [31]:
# 특정 장르(호러)로 확인
get_similar_movie(3340)

['Bride of the Monster (1956)',
 'It Came from Beneath the Sea (1955)',
 'Mole People, The (1956)',
 'It Came from Outer Space (1953)',
 'Tarantula (1955)',
 'House of Dracula (1945)',
 'Quatermass II (1957)',
 'Son of Frankenstein (1939)',
 'Hideous Sun Demon, The (1959)',
 'Thing From Another World, The (1951)']

---

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [32]:
user = user_to_idx['ryu']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.51690996),
 (4, 0.39958698),
 (23, 0.29751894),
 (248, 0.29224402),
 (48, 0.29125494),
 (81, 0.26859266),
 (121, 0.26078665),
 (545, 0.24611808),
 (322, 0.24385221),
 (812, 0.24205112),
 (51, 0.22752583),
 (222, 0.2243658),
 (110, 0.21319458),
 (450, 0.20317262),
 (85, 0.18476728),
 (139, 0.18277906),
 (269, 0.18200357),
 (513, 0.18059345),
 (626, 0.18026799),
 (33, 0.17907253)]

In [33]:
[[movie_to_title[idx_to_movie[i[0]]],i[0]] for i in movie_recommended]

[['Toy Story 2 (1999)', 50],
 ["Bug's Life, A (1998)", 4],
 ["Schindler's List (1993)", 23],
 ['Good Will Hunting (1997)', 248],
 ['Saving Private Ryan (1998)', 48],
 ['Green Mile, The (1999)', 81],
 ['Silence of the Lambs, The (1991)', 121],
 ['Santa Clause, The (1994)', 545],
 ['Babe (1995)', 322],
 ['Nutty Professor, The (1996)', 812],
 ['Fargo (1996)', 51],
 ['Pulp Fiction (1994)', 222],
 ['Groundhog Day (1993)', 110],
 ['Life Is Beautiful (La Vita è bella) (1997)', 450],
 ['League of Their Own, A (1992)', 85],
 ['October Sky (1999)', 139],
 ['GoodFellas (1990)', 269],
 ['Jumanji (1995)', 513],
 ["You've Got Mail (1998)", 626],
 ['Aladdin (1992)', 33]]

In [34]:
glass_bottom_boat = movie_to_idx[3144]
explain = als_model.explain(user, csr_data, itemid=glass_bottom_boat)

In [36]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]
# [[movie_to_title[idx_to_movie[i[0]]], i[0]] for i in explain[1]]

[(2431, 0.01554583337160987),
 (1, -0.0004748165193719286),
 (318, -0.0021620350034421906),
 (3156, -0.0039290056052668325),
 (367, -0.006776902196134611)]

---

### 루브릭 평가

__1. CSR matrix가 정상적으로 만들어졌다.__  
_(사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.)_

- 6040x3628로 정상적으로 만들어졌다.

__2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.__  
_(사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.)_

- 내가 좋아하는 영화(mask)의 내적수치 : 0.40706974
- 내가 좋아하지 않는 영화(scream)의 내적수치 : 0.026015326
- 좋아하는 영화가 1이 나오지는 않았지만 좋아하지 않는 영화보다 월등히 높게 나왔다.

__3. 비슷한 영화찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.__  
_(MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.)_

- 내적 수치 또는 추천된 영화를 보았을 때 유사도가 높아 보이나, 기여도 측면에서 매우 낮게 측정되거나 마이너스 값을 보여 측정에 정확도에 대하여 의구심이 든다.

---

## 회고

__1. 프로젝트 진행시 어려웠던 점__

- 영화 아이디와 영화 이름를 연결하는 데이터 전처리

__2. 프로젝트를 진행하면서 알아낸 점 또는 아직 모호한 점__

- 기여도가 정삭적으로 측정된 것인가?
- 만약 맞다면 왜 저렇게 나왔는가?

__3. 루브릭 평가 지표를 맞추기 위해 시도한 점__

- 기여도를 높이기 위해 비슷한 장르의 영화를 선택했지만 해결하지 못했다.

__4. 루브릭 평가 지표를 달성하지 못한 이유__

- 기여도가 어떻게 작용되는지 이해하지 못한 것 같다.

__5. 프로젝트 진행 후 느낀 점 및 다짐__

- 데이더 전처리 과정의 중요성을 다시 느꼈다.
- 넘파이나 판다스 등 기본적인 부분을 다시 공부해야 겠다.